** Cargad las tres librerías de siempre ** 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

** La data que vamos a usar en este ejercicio corresponde a varias medidas geométricas hechas sobre tres variedades de trigo. La información la podéis encontrar en __[el repositorio UCI](https://archive.ics.uci.edu/ml/datasets/seeds)__ . Cargad y cread un dataframe de pandas a partir del fichero .txt con el nombre `seeds_dataset` que os habéis descargado del repositorio del curso. Este fichero es un txt y no un csv, así  que seguramente tengáis que usar otra función que no sea `read_csv`. Podéis mirar en la documentación de __[pandas](https://pandas.pydata.org/pandas-docs/stable/io.html)__ para saber qué función usar**

In [ ]:
dat_seeds = pd.read_table('../../datasets/seeds_dataset.txt',header=None)

** Mostrad en pantalla info sobre los tipos de las columnas y de si tienen NaNs. Asimismo, mostrad en pantalla información estadística de cada columna, con su medía, quartiles...  ** 

In [ ]:
dat_seeds.info()

In [ ]:
dat_seeds.describe()

** Las siete primeras columnas corresponden a las features y la última al target. Cambiad por tanto el nombre de las columnas a ['area', 'perimeter', 'compactness', 'kernel-length', 'kernel-width', 'asymmetry-coefficient', 'kernel-groove-length', 'variety'] **

In [ ]:
dat_seeds.columns= ['area', 'perimeter', 
  'compactness', 'kernel-length', 
  'kernel-width', 'asymmetry-coefficient', 'kernel-groove-length', 
  'variety']

In [ ]:
dat_seeds.head(10)

** Cread una matrix numpy `X` con las features y un array `y` con las variedades de trigo **

In [ ]:
X = dat_seeds.drop(columns=['variety']).values
y = dat_seeds['variety'].values

** Vamos tratar algún algoritmo de clustering sobre estos datos ( la matriz X de features). Para ello, antes de nada, estandarizar esto datos usando la clase de scikit correspondiente ** 

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_scaled = ss.fit_transform(X)

** Usad Kmeans para ver cómo de buena es la reconstrucción de los clusters. Medid esto usando cualquiera de las métricas vistas en clase. Cuando defináis el objeto de la clase Kmeans, elegid `n_clusters` = 3 (puesto que sabemos que debería haber 3 clases), `random_state` = 0, para que sea reproducible y `n_init` = 100 para asegurarnos encontrar el mejor resultado posible ** 

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, completeness_score, homogeneity_score
km = KMeans(n_clusters = 3, random_state=0, n_init=100)
km.fit(X_scaled)

print("adjusted rand score = " , adjusted_rand_score(y, km.predict(X_scaled)))
print("adjusted mutual info score = " , adjusted_mutual_info_score(y, km.predict(X_scaled)))
print("completeness score = " , completeness_score(y, km.predict(X_scaled)))
print("homogeneity score = " , homogeneity_score(y, km.predict(X_scaled)))

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agg = AgglomerativeClustering(n_clusters = 3)

y_pred = agg.fit_predict(X_scaled)

print("adjusted rand score = " , adjusted_rand_score(y, y_pred))
print("adjusted mutual info score = " , adjusted_mutual_info_score(y, y_pred))
print("completeness score = " , completeness_score(y, y_pred))
print("homogeneity score = " , homogeneity_score(y, y_pred))

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components = 3, n_init=100, random_state=0)

gm.fit(X_scaled)

y_pred = gm.predict(X_scaled)
print("adjusted rand score = " , adjusted_rand_score(y, y_pred))
print("adjusted mutual info score = " , adjusted_mutual_info_score(y, y_pred))
print("completeness score = " , completeness_score(y, y_pred))
print("homogeneity score = " , homogeneity_score(y, y_pred))

** Usad el método elbow para mostrar que en efecto el número de clusters óptimo es 3. Acordaos de que el atributo *inertia_* nos daba la información que necesitamos saber y plotear una vez fitteado el kmeans. **

In [ ]:
res = []
for k in np.arange(1,10):

    km = KMeans(n_clusters = k, random_state=0)
    km.fit(X_scaled)
    res.append(km.inertia_)

plt.plot( np.arange(1,10), res)

In [ ]:
np.corrcoef(X.T).shape

** (Opcional) Calculad la matriz de correlaciones entre las features y plotead esta matriz. Para calcular la matriz de correlaciones podeís usar la función `corrcoef` de numpy o usar el método `corr` que tienen los data frames de pandas. Cuidado porque si usáis la función de numpy, la correlación la hace entre las filas y a nosotros nos interesan las columnas, esto es, los features, así que en este caso o bien cuando useis `corrcoef` tenéis que elegir el argumento `rowvar` = False o transponer la matriz antes de llamar a este método. Usad la matriz de features sin estandarizar. Para plotear esta matriz de correlaciones, podéis usar el método `imshow` de matplotlib **

In [ ]:
plt.imshow(np.corrcoef(X, rowvar=False))

In [ ]:
plt.imshow(dat_seeds.iloc[:,:7].corr())

** Realizad una pca sobre la matriz de features (estandarizada, obviamente), de tal manera que nos dé aquellas componentes que expliquen el 95 % de la varianza de los datos. Transformad los datos originales al espacio de las componentes seleccionadas por la PCA **

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
X_pca = pca.fit_transform(X_scaled)

** Haced un scatter plot mostrando los puntos correspondientes a las dos primeras componentes. Acordaos de que para hacer un scatter plot tenéis que usar la función `scatter` de matplotlib. Pasadle como argumento `c` el vector de targets y. ** 

In [ ]:
plt.scatter(X_pca[:,0],X_pca[:,1], c=y)

In [ ]:
plt.scatter(X_pca[:,0],X_pca[:,2], c=dat_seeds.iloc[:,7])

In [ ]:
plt.scatter(X_pca[:,1],X_pca[:,2], c=dat_seeds.iloc[:,7])

** Mostrad de nuevo el rendimiento del clustering usando Kmeans, pero ahora sobre la matriz de datos transformados por la PCA al espacio de las componentes principales **

In [ ]:
km = KMeans(n_clusters = 3, random_state=0, n_init=100)
km.fit(X_pca)
y_pred = km.predict(X_pca)
print("adjusted rand score = " , adjusted_rand_score(y, y_pred))
print("adjusted mutual info score = " , adjusted_mutual_info_score(y, y_pred))
print("completeness score = " , completeness_score(y, y_pred))
print("homogeneity score = " , homogeneity_score(y, y_pred))

In [ ]:
gm = GaussianMixture(n_components = 3, n_init=100, random_state=0)
gm.fit(X_pca)
y_pred = gm.predict(X_pca)

print("adjusted rand score = " , adjusted_rand_score(y, y_pred))
print("adjusted mutual info score = " , adjusted_mutual_info_score(y, y_pred))
print("completeness score = " , completeness_score(y, y_pred))
print("homogeneity score = " , homogeneity_score(y, y_pred))

** Uno de los atributos del objeto de la pca cuando lo hemos fitteado es `components_`, que muestra el peso de cada feature en cada componente. Plotead el output de llamar a este atributo usando el metodo `imshow` de matplotlib**

In [ ]:
plt.imshow(np.abs(pca.components_), cmap = plt.cm.bwr)
#plt.imshow(pca.components_)
plt.colorbar()